选择三秒以后／实验中后期的数据


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import utils
import load_data

seed = 0
utils.set_random_seed(seed)
PATH = '~/Desktop/Development/Python/dataset/data_preprocessed_python/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pdata  = pd.read_csv(PATH + 'preprocessed_data_1d.csv')
plabel = pd.read_csv(PATH + 'preprocessed_label_1d.csv')
ndata = np.array(pdata)
ndata = ndata.reshape(40*32, 40, 8064)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
def label_map(x, pos=0, classes=2):
    if x[pos] >= 5:
        return [1, 0]
    else:
        return [0, 1]
    
def get_dataloader(file_path, batch_size=64, test_size=0.2, task='classify'):
    
    pdata  = pd.read_csv(file_path + 'preprocessed_data_1d.csv')
    plabel = pd.read_csv(file_path + 'preprocessed_label_1d.csv')
    ndata = np.array(pdata)
    ndata = ndata.reshape(40*32, 40, 8064)
    if task == 'classify':
        nlabel = np.array(plabel.apply(label_map, axis=1))
    else:
        nlabel = np.array(plabel)
    train_data, test_data, train_label, test_label = train_test_split(ndata, nlabel, test_size=0.20)
    train_set = MyDataset(train_data, train_label)
    test_set  = MyDataset(test_data, test_label)
    
    train_loader = DataLoader(train_set, batch_size=batch_size)
    test_loader  = DataLoader(test_set , batch_size=batch_size)
    
    return train_loader, test_loader

def boost_dataloader(data, label, batch_size=64):
    
    train_data, test_data, train_label, test_label = train_test_split(data, label, test_size=0.20)
    train_set = MyDataset(train_data, train_label)
    test_set  = MyDataset(test_data, test_label)
    
    train_loader = DataLoader(train_set, batch_size=batch_size)
    test_loader  = DataLoader(test_set , batch_size=batch_size)
    
    return train_loader, test_loader

    
class MyDataset(Dataset):
    def __init__(self, data, label, task='classify'):
        self.data  = data
        self.label = label
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.task = task
        if task not in ['classify', 'regression']:
            raise ValueError("UNDEFINED TASK")
            
    def __len__(self):
        return self.label.shape[0]
        
    def __getitem__(self, index):
        data  = self.data[index]
        label = self.label[index]
        
        data  = torch.FloatTensor(data).to(self.device)
        if self.task == 'classify':
            label = torch.LongTensor(label).to(self.device)
        else:
            label = torch.FloatTensor(label).to(self.device)
        return data, label
        
#train_loader, test_loader = get_dataloader(PATH)

In [4]:
def label_map_plus(x, pos=0, classes=2):
    if classes == 3:
        if x[pos] >= 7:
            return [1, 0, 0]
        elif x[pos] >= 4:
            return [0, 1, 0]
        else:
            return [0, 0, 1]
    else:
        if x[pos] >= 5:
            return [1, 0]
        else:
            return [0, 1]
nlabel = np.array(plabel.apply(label_map_plus, axis=1))
train_loader, test_loader = boost_dataloader(ndata, nlabel)

In [5]:
nlabel[:10]

array([list([1, 0]), list([1, 0]), list([1, 0]), list([0, 1]),
       list([1, 0]), list([1, 0]), list([1, 0]), list([1, 0]),
       list([0, 1]), list([0, 1])], dtype=object)

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class ResidualBlock1d(nn.Module):
    def __init__(self,in_channel, out_channel, kernel_size=3, stride=1, padding=1,dilation=1):
        super(ResidualBlock1d, self).__init__()
        self.block = nn.Sequential(
            nn.BatchNorm1d(in_channel),
            nn.ReLU(),
            nn.Conv1d(in_channel, out_channel, kernel_size, stride, padding, dilation),
            nn.BatchNorm1d(out_channel),
            nn.ReLU(),
            nn.Conv1d(out_channel,out_channel, kernel_size, stride, padding, dilation),
        )

    def forward(self, x):
        out = self.block(x)
        out += x
        return out

class ResidualBlock2d(nn.Module):
    def __init__(self,in_channel, out_channel, kernel_size=3, stride=1, padding=1,dilation=1):
        super(ResidualBlock2d, self).__init__()
        self.block = nn.Sequential(
            nn.BatchNorm2d(in_channel),
            nn.ReLU(),
            nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, dilation, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(),
            nn.Conv1d(out_channel,out_channel, kernel_size, stride, padding, dilation, bias=False),
        )

    def forward(self, x):
        out = self.block(x)
        out += x
        return out
        
class PureCNN(nn.Module):
    
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=40 , out_channels=128, kernel_size=8, stride=4)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=4, stride=2)
        
        self.max_pool1 = nn.MaxPool1d(kernel_size=2)
        self.max_pool2 = nn.MaxPool1d(kernel_size=2)
        
        self.block1 = ResidualBlock1d(128, 128, 3, padding=1)
        self.block2 = ResidualBlock1d(128, 128, 3, padding=1)
        self.block3 = ResidualBlock1d(128, 128, 3, padding=1)
        self.block4 = nn.Conv1d(in_channels=128, out_channels=8, kernel_size=3, stride=1)
        
        self.fn1 = nn.Linear(1992, 1024)
        self.fn2 = nn.Linear(1024, 2)
        
        self.bn1 = nn.BatchNorm1d(40)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(128)
        
        self.drop1 = nn.Dropout(0.3)
        self.drop2 = nn.Dropout(0.3)
        self.drop3 = nn.Dropout(0.3)
        
    def forward(self, x):
        
        #x = x[:, :32, 2048:]
        x = self.bn1(x)
        x = self.conv1(x)
        x = F.relu(self.bn2(x))
        x = self.max_pool1(x)
        
        x = self.bn2(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        
        x = x.view(-1, 1992)
        x = F.relu(self.fn1(x))
        x = F.softmax(self.fn2(x), -1)
        return x
    
class ConvLSTM(nn.Module):
    
    def __init__(self):
        super(ConvLSTM, self).__init__()
        
        self.lstm = nn.LSTM(40, 512, 1)
        self.fn1 = nn.Linear(hidden_size, 512)
        self.fn2 = nn.Linear(512, 2)
        
    def forward(self, x):
        
        h_0 = torch.zeros(1, x, 512)
        c_0 = torch.zeros(1, x, 512)
        
        out = self.lstm(x,(h_0, c_0))
        out = F.relu(self.fn1(out))
        out = self.fn2(out)
        return out
    
        
class SubConvNet(nn.Module):
    
    def __init__(self, in_channel, hidden_channel=64, out_channel=2):
        super(SubConvNet, self).__init__()
        
        self.conv = nn.Conv1d(in_channel, hidden_channel, kernel_size=4, stride=2)
        self.block1 = ResidualBlock1d(hidden_channel, hidden_channel)
        self.block2 = ResidualBlock1d(hidden_channel, hidden_channel)
        self.conb = nn.Conv1d(hidden_channel, out_channel, 1)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.conb(x)
        x = x.view(batch_size, -1)
        return x
    
class AttentionAwareNet(nn.Module):
    
    def __init__(self, hidden_size):
        super(AttentionAwareNet, self).__init__()
        self.q_ = nn.Linear()
        
    def forward(self, x):
        
        
        attn = F.softmax(score, -1)
        out = torch.matmul(attn, x)
        return out
    
    def attention_net(self, lstm_output):
        #print(lstm_output.size()) = (squence_length, batch_size, hidden_size*layer_size)

        output_reshape = torch.Tensor.reshape(lstm_output, [-1, self.hidden_size*self.layer_size])
        #print(output_reshape.size()) = (squence_length * batch_size, hidden_size*layer_size)

        attn_tanh = torch.tanh(torch.mm(output_reshape, self.w_omega))
        #print(attn_tanh.size()) = (squence_length * batch_size, attention_size)

        attn_hidden_layer = torch.mm(attn_tanh, torch.Tensor.reshape(self.u_omega, [-1, 1]))
        #print(attn_hidden_layer.size()) = (squence_length * batch_size, 1)

        exps = torch.Tensor.reshape(torch.exp(attn_hidden_layer), [-1, self.sequence_length])
        #print(exps.size()) = (batch_size, squence_length)

        alphas = exps / torch.Tensor.reshape(torch.sum(exps, 1), [-1, 1])
        #print(alphas.size()) = (batch_size, squence_length)

        alphas_reshape = torch.Tensor.reshape(alphas, [-1, self.sequence_length, 1])
        #print(alphas_reshape.size()) = (batch_size, squence_length, 1)

        state = lstm_output.permute(1, 0, 2)
        #print(state.size()) = (batch_size, squence_length, hidden_size*layer_size)

        attn_output = torch.sum(state * alphas_reshape, 1)
        #print(attn_output.size()) = (batch_size, hidden_size*layer_size)

        return attn_output
    
class AttentionNet(nn.Module):
    
    def __init__(self, 
                 time_lens=63, 
                 input_size=126, 
                 hidden_size=256, 
                 output_size=2, 
                 layer_size=1, 
                 batch_size=64,
                 bidirectional=False
                ):
        super(AttentionNet, self).__init__()
        
        self.time_lens = time_lens
        self.input_size  = input_size
        self.hidden_size = hidden_size
        self.layer_size  = layer_size
        self.batch_size  = batch_size
        self.device = torch.device('cuda')
        
        self.subconv = SubConvNet(in_channel=40)
        
        self.lstm = nn.LSTM(input_size, hidden_size, layer_size, bidirectional=bidirectional)
        #self.attn = AttentionAwareNet()
        self.maxpool = nn.MaxPool1d(time_lens)
        self.fn = nn.Linear(hidden_size * layer_size, output_size)

    def forward(self, x):
        
        x = x.chunk(self.time_lens, 2)
        x = torch.stack(x, 1)
        x = x.reshape(self.batch_size * self.time_lens, 40, 128)
        x = self.subconv(x)
        
        x = x.view(self.batch_size, self.time_lens, self.input_size)
        x = x.permute(1, 0, 2)
        
        h_0 = torch.zeros(self.layer_size, self.batch_size, self.hidden_size).to(self.device)
        c_0 = torch.zeros(self.layer_size, self.batch_size, self.hidden_size).to(self.device)
        
        x, (h_final, c_final) = self.lstm(x, (h_0, c_0))
        x = x.permute(1, 2, 0)
        x = self.maxpool(x)
        x = x.view(self.batch_size, -1)
        x = F.softmax(self.fn(x), -1)
        
        return x
    
class AttentionFeatureAwareNet(nn.Module):
    
    def __init__(self,
                hidden_size,
                out_size):
        
        self.cnet = SubConvNet()
        self.attn = AttentionAwareNet()
        self.fn = nn.Linear(hidden_size, out_size)
        
    def forward(self, x):
        
        x = x.view(self.batch_size * self.channel_size, -1)
        x = self.cnet(x)
        x = x.view(self.batch_size, self.channel_size, -1)
        x, score = self.attn(x)
        x = self.fn(x)
        return x, score
        
class bilstm_attn(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embed_dim, bidirectional, dropout, use_cuda, attention_size, sequence_length):
        super(bilstm_attn, self).__init__()

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.bidirectional = bidirectional
        self.dropout = dropout
        self.use_cuda = use_cuda
        self.sequence_length = sequence_length
        self.lookup_table = nn.Embedding(self.vocab_size, self.embed_dim, padding_idx=const.PAD)
        self.lookup_table.weight.data.uniform_(-1., 1.)

        self.layer_size = 1
        self.lstm = nn.LSTM(self.embed_dim,
                            self.hidden_size,
                            self.layer_size,
                            dropout=self.dropout,
                            bidirectional=self.bidirectional)

        if self.bidirectional:
            self.layer_size = self.layer_size * 2
        else:
            self.layer_size = self.layer_size

        self.attention_size = attention_size
        if self.use_cuda:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size).cuda())
            self.u_omega = Variable(torch.zeros(self.attention_size).cuda())
        else:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size))
            self.u_omega = Variable(torch.zeros(self.attention_size))

        self.label = nn.Linear(hidden_size * self.layer_size, output_size)

    # self.attn_fc_layer = nn.Linear()

    def attention_net(self, lstm_output):
        #print(lstm_output.size()) = (squence_length, batch_size, hidden_size*layer_size)

        output_reshape = torch.Tensor.reshape(lstm_output, [-1, self.hidden_size*self.layer_size])
        #print(output_reshape.size()) = (squence_length * batch_size, hidden_size*layer_size)

        attn_tanh = torch.tanh(torch.mm(output_reshape, self.w_omega))
        #print(attn_tanh.size()) = (squence_length * batch_size, attention_size)

        attn_hidden_layer = torch.mm(attn_tanh, torch.Tensor.reshape(self.u_omega, [-1, 1]))
        #print(attn_hidden_layer.size()) = (squence_length * batch_size, 1)

        exps = torch.Tensor.reshape(torch.exp(attn_hidden_layer), [-1, self.sequence_length])
        #print(exps.size()) = (batch_size, squence_length)

        alphas = exps / torch.Tensor.reshape(torch.sum(exps, 1), [-1, 1])
        #print(alphas.size()) = (batch_size, squence_length)

        alphas_reshape = torch.Tensor.reshape(alphas, [-1, self.sequence_length, 1])
        #print(alphas_reshape.size()) = (batch_size, squence_length, 1)

        state = lstm_output.permute(1, 0, 2)
        #print(state.size()) = (batch_size, squence_length, hidden_size*layer_size)

        attn_output = torch.sum(state * alphas_reshape, 1)
        #print(attn_output.size()) = (batch_size, hidden_size*layer_size)

        return attn_output

    def forward(self, input_sentences, batch_size=None):
        input = self.lookup_table(input_sentences)
        input = input.permute(1, 0, 2)

        if self.use_cuda:
            h_0 = Variable(torch.zeros(self.layer_size, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(self.layer_size, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(self.layer_size, self.batch_size, self.hidden_size))
            c_0 = Variable(torch.zeros(self.layer_size, self.batch_size, self.hidden_size))

        lstm_output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        attn_output = self.attention_net(lstm_output)
        logits = self.label(attn_output)
        return logits
    
    
class ScaledDotProductAttention(nn.Module): # 点乘
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask): # 实现注意力公式
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        scores.masked_fill_(attn_mask, -1e9)
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module): # 多头注意力
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
   
    def forward(self, Q, K, V, attn_mask):
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn
    
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)

    def forward(self, inputs):
        residual = inputs # inputs : [batch_size, len_q, d_model]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        return nn.LayerNorm(d_model)(output + residual)

In [86]:
from tensorboardX import SummaryWriter
#net = PureCNN().to(device)
net = AttentionNet().to(device)
criterion_mse = nn.MSELoss()
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-2)
writer = SummaryWriter("runs/ConvLSTM_Maxpool-2-classify_" + str(datetime.datetime.now()))

for i in range(50):
    train_correct = train_total = 0
    test_correct  = test_total  = 0
    train_loss = 0
    
    net.train()
    for input, label in train_loader:
        output = net(input)
        prediction = torch.argmax(output, 1)
        label = torch.argmax(label, 1)
        
        loss = criterion_cel(output, label)
        train_loss += loss.item()
        
        train_correct += (prediction == label).sum().float()
        train_total += len(label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()
    for input, label in test_loader:
        output = net(input)
        prediction = torch.argmax(output, 1)
        label = torch.argmax(label, 1)
        
        test_correct += (prediction == label).sum().float()
        test_total += len(label)
        
    print('e', i)
    writer.add_scalar('loss', train_loss, i)
    writer.add_scalar('accuracy/train', train_correct/train_total, i)
    writer.add_scalar('accuracy/test', test_correct /test_total, i)
#     print("="*20)
#     print("epoch:",i)
#     print(" train acc:{:.4f}, loss:{:.4f}".format(train_correct/train_total, train_loss))
#     print(" test  acc:{:.4f}".format(test_correct /test_total))
writer.close()
print('finish')

e 0
e 1
e 2
e 3
e 4
e 5
e 6
e 7
e 8
e 9
e 10
e 11
e 12
e 13
e 14
e 15
e 16
e 17
e 18
e 19
e 20
e 21
e 22
e 23
e 24
e 25
e 26
e 27
e 28
e 29
e 30
e 31
e 32
e 33
e 34
e 35
e 36
e 37
e 38
e 39
e 40
e 41
e 42
e 43
e 44
e 45
e 46
e 47
e 48
e 49
finish


In [83]:
test_net = SubConvNet()

In [84]:
val = torch.randn(4, 40, 128)
out = test_net(val)
out.shape

torch.Size([4, 126])

In [69]:
out

tensor([[0.5136, 0.4864],
        [0.5034, 0.4966],
        [0.5104, 0.4896],
        [0.5109, 0.4891]], device='cuda:0', grad_fn=<SoftmaxBackward>)